In [ ]:
import warnings
import os
os.chdir(f'/mnt/primary/exposure-fairness/')
import config
warnings.filterwarnings('ignore')
import pyterrier as pt
if not pt.started():
    pt.init()

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 100)
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator

In [ ]:
# bm25 = df[(df['modelname'] == 'bm25') & (df['threshold'] == 0)]
# gini_bm25 = df.loc[df['Model'] == 'BM25',['Gini','Gini.1','Gini.2','Gini.3']]

In [ ]:
csv = f'/mnt/primary/exposure-fairness/results/result_all_models_all_groups.csv'
df = pd.read_csv(csv, index_col=0).reset_index()

In [ ]:

csv = f'{config.data_dir}/v1//mnt/datasets/cxj/exposure-fairness/v2'
df2 = pd.read_pickle(csv).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df[(df['modelname'] == 'bm25') & (df['threshold'] == 0)]

In [14]:
csv = f'{config.data_dir}/v2/bm25_msmarco-passage_dev_100.csv'
df3 = pd.read_pickle(csv).reset_index(drop=True)

In [15]:
df3.head()

,qid,docid,docno,score,rank,query
0,1048578,7187236,7187236,67.528593,0,cost of endless pools swim spa
1,1048578,7471198,7471198,62.121928,1,cost of endless pools swim spa
2,1048578,5365326,5365326,51.958492,2,cost of endless pools swim spa
3,1048578,7187241,7187241,51.947947,3,cost of endless pools swim spa
4,1048578,6802210,6802210,47.925985,4,cost of endless pools swim spa


In [17]:
! pip list | grep numpy

numpy                     1.26.4


In [ ]:
grouped_df = df2.groupby('modelname')

In [ ]:
grouped_df.get_group(('bm25',500))

In [ ]:
def sub_plot(plt,threshold,title,col, ylabel):
    # bm25 = df[(df['modelname'] == 'bm25') & (df['threshold'] == threshold)]
    # bm25_monot5 = df[(df['modelname'] == 'bm25_monot5') & (df['threshold'] == threshold)]
    # splade = df[(df['modelname'] == 'splade') & (df['threshold'] == threshold)]
    # tctcolbert = df[(df['modelname'] == 'tctcolbert') & (df['threshold'] == threshold)]
    # bm25_tctcolbert = df[(df['modelname'] == 'bm25_tctcolbert') & (df['threshold'] == threshold)]

    bm25 = df[(df['modelname'] == 'bm25') & (df['threshold'] == threshold)]
    bm25_monot5 = df[(df['modelname'] == 'bm25_monot5') & (df['threshold'] == threshold)]
    splade = df[(df['modelname'] == 'splade') & (df['threshold'] == threshold)]
    tctcolbert = df[(df['modelname'] == 'tctcolbert') & (df['threshold'] == threshold)]
    bm25_tctcolbert = df[(df['modelname'] == 'bm25_tctcolbert') & (df['threshold'] == threshold)]

    x = [1,2,3,4,5]
    # col = 'min_gini'
    y = bm25[col]
    line1, = plt.plot(x,y,label='BM25', marker='.')
    y = bm25_monot5[col]
    line2, = plt.plot(x,y,label='BM25>>Mono-T5', marker='.')
    y = splade[col]
    line3, = plt.plot(x,y,label='SPLADE', marker='.')
    y = tctcolbert[col]
    line4, = plt.plot(x,y,label='TCT-ColBERT', marker='.')
    y = bm25_tctcolbert[col]
    line5, = plt.plot(x,y,label='BM25>>TCT-ColBERT', marker='.')

    # plt.set_title('Exposure Fairness')
    plt.set_title(title, fontsize=16)
    plt.set_xlabel(r'#Groups ($K$)', fontsize=16)
    
    if col == 'min_gini':
        # ylabel = 'Min Gini Coefficient'
        plt.set_ylabel(ylabel, fontsize=16)
    plt.set_xticks(ticks = x, labels = ['500', '1000', '2000', '5000', '10000'], fontsize=16)
    # plt.set_yticks(ticks = y, labels = [f'{i:.2f}' for i in y])
    return line1, line2,line3,line4,line5

In [ ]:
fig, (ax0,ax1,ax2) = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
# plt.subplots_adjust(hspace=1, wspace=0.2, left=0.1)
plt.subplots_adjust(hspace=1, wspace=0.2, left=0, right=1, bottom=0)

ylabel = 'Min (Gini), Avg (Gini), Max (Gini)'

col = 'min_gini'
ax0.xaxis.set_major_locator(MultipleLocator(1))
ax0.yaxis.set_major_locator(MultipleLocator(0.03))
# ax0.set_xlim(1,5)
ax0.set_ylim(0.12,0.20)
g0 = sub_plot(ax0,0,None,col, ylabel)

col = 'mean_gini'
ax1.xaxis.set_major_locator(MultipleLocator(1))
ax1.yaxis.set_major_locator(MultipleLocator(0.03))
# ax1.set_xlim(1,5)
ax1.set_ylim(0.22,0.30)
g1 = sub_plot(ax1,0,None,col, ylabel)

col = 'max_gini'
ax2.xaxis.set_major_locator(MultipleLocator(1))
ax2.yaxis.set_major_locator(MultipleLocator(0.05))
# ax2.set_xlim(1,5)
ax2.set_ylim(0.47,0.75)
g2 = sub_plot(ax2,0,None,col, ylabel)

g = list(g0 + g1 + g2)

fig.legend(handles=g0, loc='upper center', bbox_to_anchor=(0.5, 1.12), ncol=10, fontsize=16)
plt.savefig(f'{config.prog_dir}/aggr_gini.pdf', format="pdf", bbox_inches="tight", pad_inches=0)
plt.show()